<a href="https://colab.research.google.com/github/KarinaBelch/IC-Teste-Automatizado-de-Uniformidade-PET/blob/main/IC_Karina_Catuna.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Teste Automatizado de Uniformidade para Instrumentação de Positron Emission Tomography (PET)

In [ ]:
# @title Importando Bibliotecas

# Instalação de pacotes
!pip install pynrrd SimpleITK pydicom

# Manipulação de dados e arrays
import numpy as np
import pandas as pd

# Leitura e manipulação de arquivos médicos
# import nrrd
import SimpleITK as sitk
import pydicom

# Manipulação de arquivos e compressão
import os
import zipfile
from io import BytesIO

# Visualização
import matplotlib.pyplot as plt

# Widgets interativos
from ipywidgets import interact, widgets

# Processamento de imagens
from PIL import Image
from skimage import color, img_as_ubyte, img_as_float
from skimage.feature import canny
from skimage.transform import hough_circle, hough_circle_peaks, hough_ellipse
from skimage.draw import circle_perimeter, ellipse_perimeter
from skimage.draw import polygon
from skimage.draw import disk

# Processamento morfológico e preenchimento
from scipy import ndimage

In [ ]:
# @title Importando Google Drive
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title Obtendo arquivo DICOM

dicom_dir = "/content/drive/MyDrive/IC2024/Aquis1"
dicom_files = []

for root, dirs, files in os.walk(dicom_dir):
    for file in files:
        if file.endswith(".dcm"):
            dicom_files.append(os.path.join(root, file))

print("Arquivos DICOM encontrados:", len(dicom_files))

Arquivos DICOM encontrados: 45


In [ ]:
# @title Ordenando as fatias

# Ordenar por InstanceNumber (ordem axial)
slices = [pydicom.dcmread(f) for f in dicom_files]
slices = [s for s in slices if hasattr(s, 'InstanceNumber')]
slices.sort(key=lambda s: s.InstanceNumber)

# Converter para volume 3D
volume = np.stack([s.pixel_array for s in slices])

print("Volume 3D:", volume.shape)  # (profundidade, altura, largura)

Volume 3D: (45, 144, 144)


In [ ]:
# @title Plotando fatias

def show_slice(i):
    plt.figure(figsize=(5,5))
    plt.imshow(volume[i], cmap='gray')
    plt.title(f"Slice {i}")
    plt.axis("off")
    plt.show()

interact(show_slice, i=widgets.IntSlider(min=0, max=volume.shape[0]-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='i', max=44), Output()), _dom_classes=('widget-interact',…

<function __main__.show_slice(i)>

In [ ]:
# @title Função para Criar a Máscara
def funcMascaraCircularReduzida(image_rgb, scale):

    edges = canny(image_rgb, sigma=5, low_threshold=0.1, high_threshold=0.2)

    hough_radii = np.arange(0, image_rgb.shape[0], 1)
    hough_res = hough_circle(edges, hough_radii)

    accums, cx, cy, radii = hough_circle_peaks(hough_res, hough_radii, total_num_peaks=1)

    if len(cx) == 0:
        empty_mask = np.zeros_like(image_rgb, dtype=bool)
        empty_edges = np.zeros_like(image_rgb, dtype=bool)
        return empty_edges, empty_mask, 0, 0, 0

    # Cria círculo com raio reduzido
    r_reduzido = int(radii[0] * scale)
    rr, cc = disk((cy[0], cx[0]), r_reduzido, shape=image_rgb.shape)

    mask = np.zeros_like(image_rgb, dtype=bool)
    mask[rr, cc] = True

    return edges, mask, r_reduzido, cx[0], cy[0]

In [ ]:
# @title Declarando os arrays
# Prepara array para armazenar volume preenchido

edges_volume = np.zeros_like(volume, dtype=np.uint8)
filled_volume = np.zeros_like(volume, dtype=np.uint8)
raio_volume = np.zeros(len(dicom_files))
cx_volume = np.zeros(len(dicom_files))
cy_volume = np.zeros(len(dicom_files))

In [ ]:
# @title Populando os arrays
for i in range(volume.shape[0]):

    image = volume[i]
    image_rgb = img_as_float(image)

    edges, filled, raio, cx, cy = funcMascaraCircularReduzida(image_rgb, scale=0.9)
    edges_volume[i] = edges
    filled_volume[i] = filled
    raio_volume[i] = raio
    cx_volume[i] = cx
    cy_volume[i] = cy

In [ ]:
# @title Criando a máscara
imagem_mascara = filled_volume*volume

In [ ]:
# @title Plotando gráficos com as imagens
def show_slice(i):
    image = volume[i]

    image_rgb = img_as_float(image)
    edges_rgb = img_as_float(edges_volume[i])
    filled_rgb = img_as_float(filled_volume[i])
    imagem_mascara

    fig, axs = plt.subplots(1, 4, figsize=(15, 10))

    axs[0].imshow(image_rgb, cmap='gray')
    axs[0].set_title(f'Imagem Original (slice {i})')
    axs[0].axis('off')

    axs[1].imshow(edges_rgb, cmap='gray')
    axs[1].set_title('Borda')
    axs[1].axis('off')

    axs[2].imshow(filled_rgb, cmap='gray')
    axs[2].set_title('Borda Reduzida e Preenchida (90%)')
    axs[2].axis('off')

    axs[3].imshow(imagem_mascara[i], cmap='gray')
    axs[3].set_title('Imagem com a máscara aplicada')
    axs[3].axis('off')

    plt.show()

interact(show_slice, i=widgets.IntSlider(min=0, max=volume.shape[0]-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='i', max=44), Output()), _dom_classes=('widget-interact',…

<function __main__.show_slice(i)>

In [ ]:
# @title Função para retirar espaços em branco da imagem
def recorta_por_circulo(image, cx, cy, raio):
    h, w = image.shape[:2]

    if (h == 0 or w == 0):
        return np.zeros_like(image)

    # Limites do retângulo
    x_min = max(int(cx - raio + 1), 0)
    x_max = min(int(cx + raio), w)
    y_min = max(int(cy - raio + 1), 0)
    y_max = min(int(cy + raio), h)

    if y_max <= y_min or x_max <= x_min:
      return np.zeros_like(image)

    # Recorta
    recorte = image[y_min:y_max, x_min:x_max]

    return recorte

In [ ]:
# @title Chamando a função

imagem_cortada_volume = []

for i in range(len(volume)):
    imagem_cortada = recorta_por_circulo(imagem_mascara[i], cx_volume[i], cy_volume[i], raio_volume[i])
    imagem_cortada_volume.append(imagem_cortada)

In [ ]:
# @title Plotando gráfico
def show_slice(i):
    plt.figure(figsize=(5,5))
    plt.imshow(imagem_cortada_volume[i], cmap='gray')
    plt.title(f"Slice {i}")
    plt.show()

interact(show_slice, i=widgets.IntSlider(min=0, max=volume.shape[0]-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='i', max=44), Output()), _dom_classes=('widget-interact',…

<function __main__.show_slice(i)>

# Metodo 1 | Miller | 7 Circulos

In [ ]:
# @title Função para criar os 7 círculos
def func_Circulos(imagemCortada):
    diametro = imagemCortada.shape[0]
    nx = ny = diametro

    # Centros e distâncias
    cx = (nx // 2) + 1
    cy = (ny // 2) + 1
    dx = (nx // 3) + 1
    dy = (ny // 3) + 1

    # Raios
    rx = dx / 2
    ry = dy / 2

    x, y = np.meshgrid(np.arange(1, nx+1), np.arange(1, ny+1))

    lista_circulos = []

    for i in [-1, 0, 1]:
        Im = np.zeros((nx, ny))
        Im[((x - cx + i * 2 * rx) ** 2 + (y - cy) ** 2) < rx ** 2] = 1
        lista_circulos.append(Im)

    for i in [-1, 1]:
        Im = np.zeros((nx, ny))
        Im[((x - cx + i * rx) ** 2 + (y - cy + np.sqrt(3) * rx) ** 2) < rx ** 2] = 1
        lista_circulos.append(Im)

    for i in [-1, 1]:
        Im = np.zeros((nx, ny))
        Im[((x - cx + i * rx) ** 2 + (y - cy - np.sqrt(3) * rx) ** 2) < rx ** 2] = 1
        lista_circulos.append(Im)

    return lista_circulos

In [ ]:
# @title Aplicando a mascára para cada círculos
Im_circulos_volume = []

for i in range(len(imagem_cortada_volume)):
    circulos = func_Circulos(imagem_cortada_volume[i])  # 7 imagens
    imagens_com_circulos = [c * imagem_cortada_volume[i] for c in circulos]

    # Cria a máscara unida
    mascara_unida = np.clip(np.sum(circulos, axis=0), 0, 1)
    imagem_unida = mascara_unida * imagem_cortada_volume[i]

    imagens_com_circulos.append(imagem_unida)  # adiciona como oitava imagem
    Im_circulos_volume.append(imagens_com_circulos)

In [ ]:
# @title Plotando as máscaras para cada círculo
def show_slice(i):
    fig, axs = plt.subplots(1, 9, figsize=(20, 4))

    axs[0].imshow(imagem_cortada_volume[i], cmap='gray')
    axs[0].set_title(f'Original\n(slice {i})')
    axs[0].axis('off')

    for j in range(7):
        axs[j+1].imshow(Im_circulos_volume[i][j], cmap='gray')
        axs[j+1].set_title(f'Círculo {j+1}')
        axs[j+1].axis('off')

    axs[8].imshow(Im_circulos_volume[i][7], cmap='gray')
    axs[8].set_title("Todos unidos")
    axs[8].axis('off')

    plt.tight_layout()
    plt.show()


interact(show_slice, i=widgets.IntSlider(min=0, max=volume.shape[0]-1, step=1, value=0))

interactive(children=(IntSlider(value=0, description='i', max=44), Output()), _dom_classes=('widget-interact',…

<function __main__.show_slice(i)>

In [ ]:
# @title Gerando arquivo excel
dados = []

for i, fatia in enumerate(Im_circulos_volume):  # i = índice da slice
    for j in range(len(fatia)):  # j = índice do círculo (0 a 7, se você incluiu o unido)
        imagem_mascarada = fatia[j]

        # Pega apenas os valores dentro do círculo (ou seja, > 0)
        valores = imagem_mascarada[imagem_mascarada > 0]

        # Se não houver valores válidos (ex: tudo 0), use NaN
        if len(valores) > 0:
            mean = np.mean(valores)
            min_val = min(valores)
            max_val = max(valores)
            std = np.std(valores)
        else:
            mean = min_val = max_val = std = np.nan

        dados.append({
            "Name": f"S{i}C{j}",
            "Slice": i,
            "Circle": j + 1,  # 1 a 7 (ou 8 se unido)
            "Mean": mean,
            "Min": min_val,
            "Max": max_val,
            "Std": std
        })

df = pd.DataFrame(dados)

# Salvar como CSV ou Excel
#df.to_csv("resultados_circulos.csv", index=False)
df.to_excel("resultados_circulos.xlsx", index=False)

display(df)

,Name,Slice,Circle,Mean,Min,Max,Std
0,S0C0,0,1,NaN,NaN,NaN,NaN
1,S0C1,0,2,NaN,NaN,NaN,NaN
2,S0C2,0,3,NaN,NaN,NaN,NaN
3,S0C3,0,4,NaN,NaN,NaN,NaN
4,S0C4,0,5,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
355,S44C3,44,4,22063.275862,18945.0,26562.0,1469.425247
356,S44C4,44,5,23277.477011,18922.0,29498.0,2199.029498
357,S44C5,44,6,21784.425287,18861.0,24677.0,1298.703063
358,S44C6,44,7,22109.120690,19379.0,24711.0,1176.588231


# Metodo 2

## Para encontrar o tamanho do fisico do cilindro, obter um array com os valores da imagem (senograma)
- Encontrar número de pixels e converter em milimetros utilizando o tamanho do pixel no dicom
- Qualquer valor em dicom sempre será em mm

In [ ]:
# Tamanho do cilindro
tamanhoPixel = slices[0].PixelSpacing
print(tamanhoPixel)
alturaPixel = imagem_cortada_volume[0].shape[0]
print(alturaPixel)
alturaMM = alturaPixel * tamanhoPixel[0]
print(alturaMM)
larguraPixel = imagem_cortada_volume[0].shape[1]
print(larguraPixel)
larguraMM = larguraPixel * tamanhoPixel[1]
print(larguraMM)

[4, 4]
144
576.0
144
576.0


In [ ]:
# Obtendo o tamanho do pixel em mm

def funcQuadrados(fatia):
  y_px_mm, x_px_mm = fatia.PixelSpacing

  y_px = x_px = 12 / y_px_mm

  tamanho_total = len(imagem_cortada_volume[1])

  quadrante = np.array([y_px, x_px])
  print(quadrante)

  num_quadrados = int(tamanho_total/quadrante[0])
  num_quadrados

teste = funcQuadrados(slices[0])

[3. 3.]


In [ ]:
metodo_2 = []

def analisa_uniformidade(i, imagem_completa, tamanho_bloco=3):
    resultados = []

    for i in range(len(imagem_completa)):
      imagem = imagem_completa[i]
      h, w = imagem.shape

      for y in range(0, h, tamanho_bloco):
          for x in range(0, w, tamanho_bloco):
              bloco = imagem[y:y+tamanho_bloco, x:x+tamanho_bloco]

              # Pula blocos com todos os valores 0 (fora da área útil)
              if np.all(bloco == 0):
                  continue

              # Considera apenas os valores maiores que zero
              pixels_validos = bloco[bloco > 0]

              if pixels_validos.size == 0:
                  continue

              resultados.append({
                  "slice": i,
                  "x": x,
                  "y": y,
                  "mean": np.mean(pixels_validos),
                  "min": np.min(pixels_validos),
                  "max": np.max(pixels_validos),
                  "std": np.std(pixels_validos)
              })

    df = pd.DataFrame(resultados)
    return df

metodo_2 = analisa_uniformidade(i, imagem_cortada_volume, tamanho_bloco=3)

metodo_2.to_excel("resultados_circulos_metodo_2.xlsx", index=False)

display(metodo_2)

,slice,x,y,mean,min,max,std
0,1,9,0,19442.500000,18795,20090,647.500000
1,1,12,0,19864.833333,14718,27375,4001.268691
2,1,15,0,21056.000000,15903,27969,3129.591383
3,1,18,0,22365.222222,20677,24076,1198.345784
4,1,21,0,22983.111111,21875,25194,992.525784
...,...,...,...,...,...,...,...
7444,44,15,42,22933.666667,22695,23143,184.067620
7445,44,18,42,23733.666667,22687,24608,793.655817
7446,44,21,42,22823.000000,22043,23841,753.075472
7447,44,24,42,22460.333333,21897,23312,612.594664


Ajustes:
## Obter 8x8mm de ROI dos quadrantes do phanton

In [ ]:
mascara = sitk.BinaryThreshold(sitk.GetImageFromArray(volume), lowerThreshold=0, upperThreshold=0.1, insideValue=1, outsideValue=0)

mask_array = sitk.GetArrayFromImage(mascara)

# Streamlit

In [ ]:
!pip install -q streamlit

In [ ]:
%%writefile app.py

import streamlit as st
import streamlit as st
import pandas as pd
import numpy as np
import chardet
import matplotlib.pyplot as plt
from io import BytesIO
import nrrd
import SimpleITK as sitk
import pydicom
from ipywidgets import interact
import ipywidgets as widgets
import zipfile
import os
import shutil

from PIL import Image

# Titulo da página
st.set_page_config(page_title='Projeto de Iniciação Científica', page_icon='🥼', layout='wide')
st.title('Automated Uniformity Testing PET Instrumentation')
st.info('Projeto de Iniciação Cientifica referente ao curso de Engenharia Biomédica da Universidade Federal do ABC.')

# Menu Lateral
st.sidebar.header("Menu")
st.sidebar.caption("Leitura de arquivos DICOM.")

# Upload do arquivo
uploaded_zip = st.file_uploader(label='Upload your DICOM file:', type="zip")

if uploaded_zip:
    temp_dir = "temp_upload"

    # Limpar e criar diretório temporário
    if os.path.exists(temp_dir):
        shutil.rmtree(temp_dir)
    os.makedirs(temp_dir)

    # Salvar arquivo zip
    zip_path = os.path.join(temp_dir, "uploaded.zip")
    with open(zip_path, "wb") as f:
        f.write(uploaded_zip.getbuffer())

    # Extrair arquivos do zip
    with zipfile.ZipFile(zip_path, "r") as zip_ref:
        zip_ref.extractall(temp_dir)

    # Listar arquivos .dcm
    dicom_files = []
    for root, dirs, files in os.walk(temp_dir):
        for file in files:
            if file.endswith(".dcm"):
                dicom_files.append(os.path.join(root, file))

    st.write("Arquivos DICOM encontrados:", len(dicom_files))

    # Ler e ordenar as fatias
    slices = [pydicom.dcmread(f) for f in dicom_files if f.endswith(".dcm")]
    slices = [s for s in slices if hasattr(s, 'InstanceNumber')]
    slices.sort(key=lambda s: s.InstanceNumber)

    # Criar volume 3D
    volume = np.stack([s.pixel_array for s in slices])
    st.write("Volume 3D:", volume.shape)

    col1, col2 = st.columns(2)

    with col1:
        idx = st.slider("Escolha a fatia:", 0, volume.shape[0] - 1, 0)

    with col2:
        fig, ax = plt.subplots(figsize=(6, 6))
        ax.imshow(volume[idx], cmap="gray")
        ax.set_title(f"Slice {idx}")
        ax.axis("off")
        st.pyplot(fig)

Overwriting app.py


In [ ]:
#!npm install localtunnel

⠙⠹⠸⠼⠴⠦
up to date, audited 23 packages in 1s
⠦
⠦3 packages are looking for funding
⠦  run `npm fund` for details
⠦
2 high severity vulnerabilities

To address all issues (including breaking changes), run:
  npm audit fix --force

Run `npm audit` for details.
⠦

In [ ]:
#!npm audit fix --force

npm warn using --force Recommended protections disabled.
⠙⠹⠸⠼⠴npm warn audit Updating localtunnel to 1.8.3, which is a SemVer major change.
⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋⠙⠹⠸⠼⠴⠦⠧⠇⠏⠋npm warn deprecated cryptiles@2.0.5: This version has been deprecated in accordance with the hapi support policy (hapi.im/support). Please upgrade to the latest version to get the best features, bug fixes, and security patches. If you are unable to upgrade at this time, paid support is available for older versions (hapi.im/commercial).
⠋⠙npm warn deprecated boom@2.10.1: This version has been deprecated in accordance with the hapi support policy (hapi.im/support). Please upgrade to the latest version to get the best features, bug fixes, and security patches. If you are unable to upgrade at this time, paid support is available for older versions (hapi.im/commercial).
⠙npm warn deprecated sntp@1.0.9: This module moved to @hapi/sntp. Please make sure to switch over as this distribution i

In [ ]:
#!curl https://loca.lt/mytunnelpassword

34.66.58.158

In [ ]:
#!streamlit run app.py &>/content/logs.txt &

In [ ]:
##!npx localtunnel --port 8501

⠙⠹your url is: https://twenty-garlics-think.loca.lt
^C
